<a href="https://colab.research.google.com/github/Tanya-Sood/multimodal-gas-detection-and-classification/blob/main/DL_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
aryashah2k_multimodal_gas_detection_and_classification_path = kagglehub.dataset_download('aryashah2k/multimodal-gas-detection-and-classification')

print('Data source import complete.')


In [ ]:
import os, random, time
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models

In [ ]:
DATA_DIR = Path("/kaggle/input/multimodal-gas-detection-and-classification/Multimodal Dataset for Gas Detection and Classification")
SENSORS_CSV = DATA_DIR / "Gas Sensors Measurements" / "Gas_Sensors_Measurements.csv"
IMAGES_DIR = DATA_DIR / "Thermal Camera Images"

NUM_CLASSES = 4
BATCH_SIZE = 32
EPOCHS = 5
LR = 1e-4
IMG_SIZE = (224, 224)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv(SENSORS_CSV)

df = df.rename(columns={
    'Gas': 'label',
    'Corresponding Image Name': 'filename',
    'Serial Number': 'serial',
    'MQ2': 'mq2',
    'MQ3': 'mq3',
    'MQ5': 'mq5',
    'MQ6': 'mq6',
    'MQ7': 'mq7',
    'MQ8': 'mq8',
    'MQ135': 'mq135'
})

label_map = {lbl: i for i, lbl in enumerate(sorted(df['label'].unique()))}
df['label_idx'] = df['label'].map(label_map)

In [ ]:
class MultimodalGasDataset(Dataset):
    def __init__(self, df, images_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.images_dir = Path(images_dir)
        self.transform = transform
        self.sensor_cols = ['mq2','mq3','mq5','mq6','mq7','mq8','mq135']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path = self.images_dir / row['label'] / (str(row['filename']) + ".png")

        img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Missing image {img_path}")

        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        img = cv2.resize(img, IMG_SIZE)
        img = img.astype('float32') / 255.0
        img = np.stack([img, img, img], axis=2)
        img = torch.tensor(img.transpose(2,0,1))

        if self.transform:
            img = self.transform(img)

        sensors = torch.tensor(row[self.sensor_cols].astype(float).to_numpy(), dtype=torch.float32)
        label = int(row['label_idx'])
        return img, sensors, label

In [ ]:
transform_train = T.Compose([
    T.RandomErasing(p=0.2),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
transform_val = T.Compose([
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

In [ ]:
df_train, df_temp = train_test_split(
    df, test_size=0.30, stratify=df["label_idx"], random_state=42
)
df_val, df_test = train_test_split(
    df_temp, test_size=1/3, stratify=df_temp["label_idx"], random_state=42
)

train_ds = MultimodalGasDataset(df_train, IMAGES_DIR, transform_train)
val_ds = MultimodalGasDataset(df_val, IMAGES_DIR, transform_val)
test_ds = MultimodalGasDataset(df_test, IMAGES_DIR, transform_val)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [ ]:
class SensorOnlyMLP(nn.Module):
    def __init__(self, num_sensor_features=7, num_classes=4):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_sensor_features, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),

            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),

            nn.Linear(128, num_classes)
        )

    def forward(self, img, sensors):
        return self.model(sensors)

In [ ]:
class ImageOnlyResNet(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        backbone = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.fc = nn.Linear(512, num_classes)

    def forward(self, img, sensors):
        x = self.backbone(img)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
class MultimodalNet(nn.Module):
    def __init__(self, num_sensor_features=7, num_classes=4):
        super().__init__()

        backbone = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])

        self.sensor_mlp = nn.Sequential(
            nn.Linear(num_sensor_features, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 + 256, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, img, sensors):
        x_img = self.backbone(img)
        x_img = x_img.view(x_img.size(0), -1)
        x_s = self.sensor_mlp(sensors)
        x = torch.cat([x_img, x_s], dim=1)
        return self.classifier(x)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, preds, labels = 0, [], []

    for imgs, sensors, lbls in loader:
        imgs, sensors, lbls = imgs.to(DEVICE), sensors.to(DEVICE), lbls.to(DEVICE)

        optimizer.zero_grad()
        out = model(imgs, sensors)
        loss = criterion(out, lbls)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        preds.append(out.argmax(1).cpu().numpy())
        labels.append(lbls.cpu().numpy())

    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
    acc = accuracy_score(labels, preds)

    return running_loss / len(loader.dataset), acc

In [ ]:
def validate(model, loader, criterion):
    model.eval()
    running_loss, preds, labels = 0, [], []

    with torch.no_grad():
        for imgs, sensors, lbls in loader:
            imgs, sensors, lbls = imgs.to(DEVICE), sensors.to(DEVICE), lbls.to(DEVICE)
            out = model(imgs, sensors)
            loss = criterion(out, lbls)

            running_loss += loss.item() * imgs.size(0)
            preds.append(out.argmax(1).cpu().numpy())
            labels.append(lbls.cpu().numpy())

    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
    acc = accuracy_score(labels, preds)
    return running_loss / len(loader.dataset), acc, labels, preds

In [ ]:
def run_experiment(model, name):
    print(f"\n====== TRAINING {name} ======")

    model.to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    history = {"train_acc": [], "val_acc": [], "train_loss": [], "val_loss": []}

    for epoch in range(EPOCHS):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc, y_true, y_pred = validate(model, val_loader, criterion)

        history["train_loss"].append(tr_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(tr_acc)
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch+1}/{EPOCHS} - Train Acc: {tr_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Plot accuracy and loss
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history["train_acc"], label="Train Acc")
    plt.plot(history["val_acc"], label="Val Acc")
    plt.title(f"{name} Accuracy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history["train_loss"], label="Train Loss")
    plt.plot(history["val_loss"], label="Val Loss")
    plt.title(f"{name} Loss")
    plt.legend()
    plt.show()

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    plt.imshow(cm, cmap="Blues")
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j,i,str(cm[i,j]),ha='center',va='center')
    plt.colorbar()
    plt.show()

    return history, model

In [ ]:
results = {}

history_base, model_base = run_experiment(SensorOnlyMLP(), "Base Model (Sensor Only)")
history_tl, model_tl = run_experiment(ImageOnlyResNet(), "Transfer Learning (Image Only)")
history_hybrid, model_hybrid = run_experiment(MultimodalNet(), "Hybrid (Multimodal)")